In [1]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform
import re 
import nibabel as nib
import statistics
import sys

import mri_data
import monai_training

from reload_recursive import reload_recursive

In [2]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, Scan  # noqa: E402, F401
from mri_data import utils
from mri_data import file_manager as fm
from monai_training import preprocess  # noqa: E402
from monai_training.preprocess import DataSetProcesser

In [3]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [4]:
#! Set these variables
work_dir_name = "choroid_resegment3"

In [7]:
hostname = platform.node()
if hostname == "rhinocampus":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")

msmri_home = projects_root / "ms_mri"
training_work_dirs = drive_root / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
ensemble_out_dir = work_dir / "ensemble_output"

Scan the ensemble_out folder produced during the monai training to collect the predicted labels into a DataSet struct. Create another DataSet object with the same subjects and sessions but rooted in the main data location, then collect the manual labels 

In [8]:
ensemble_out_dataset = scan_3Tpioneer_bids(ensemble_out_dir, label="flair.t1_ensemble.nii.gz")
dataset = DataSet.dataset_like(ensemble_out_dataset)
dataset.dataroot = dataroot
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels("choroid_t1_flair", suffix_list=["CH", "ED", "DT"])
dataset = dataset_proc.dataset
dataset.sort()
ensemble_out_dataset.sort()

2024-12-15 16:50:15.529 | INFO     | monai_training.preprocess:prepare_labels:167 - Prepare Labels
 10%|█         | 1/10 [00:00<00:00,  9.84it/s]2024-12-15 16:50:15.692 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label choroid_t1_flair-CH.nii.gz for Scan(subid=1033, sesid=20171117)
2024-12-15 16:50:15.734 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label choroid_t1_flair-CH.nii.gz for Scan(subid=1119, sesid=20161010)
2024-12-15 16:50:15.841 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label choroid_t1_flair-ED.nii.gz for Scan(subid=1259, sesid=20200803)
 40%|████      | 4/10 [00:00<00:00, 13.37it/s]2024-12-15 16:50:15.871 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label choroid_t1_flair-CH.nii.gz for Scan(subid=1437, sesid=20210503)
2024-12-15 16:50:15.901 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label choroid_t1_flair-ED.nii.gz for Scan(subid=1547, sesid=20220321)
2024-12-1

Calculate Dice Scores

In [9]:
dice_scores = []
print("Dice Scores")
for scan, inference in zip(dataset, ensemble_out_dataset):
    man_label = fm.find_label(scan, "choroid_t1_flair", ["CH", "ED", "DT"])
    seg1 = nib.load(man_label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.dice_score(seg1, seg2)
    dice_scores.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(dice_scores) / len(dice_scores)))
print("Std: {:0.2f}".format(statistics.stdev(dice_scores)))

print(dice_scores)

Dice Scores
1010: 0.76
1033: 0.78
1119: 0.77
1259: 0.73
1437: 0.79
1547: 0.79
2081: 0.79
2126: 0.83
2146: 0.70
2187: 0.80
----------
Mean: 0.77
Std: 0.04
[0.7629950495049505, 0.7794253938832252, 0.7748743718592965, 0.730551478297337, 0.7935744727352202, 0.7920557284719134, 0.7911308203991131, 0.829092599047287, 0.698946169182569, 0.7952329360780065]


Calculate Hausdorff Distance for Choroid

In [10]:
haus = []
print("Hausdorff Dist")
for scan, inference in zip(dataset, ensemble_out_dataset):
    man_label = fm.find_label(scan, "choroid_t1_flair", ["CH", "ED", "DT"])
    seg1 = nib.load(man_label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.hausdorff_dist(seg1, seg2, seg2_val=1)
    haus.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(haus) / len(haus)))
print("Std: {:0.2f}".format(statistics.stdev(haus)))

print(haus)

Hausdorff Dist
1010: 1.41
1033: 1.41
1119: 1.41
1259: 1.00
1437: 2.24
1547: 2.83
2081: 1.41
2126: 1.00
2146: 4.68
2187: 1.00
----------
Mean: 1.84
Std: 1.16
[1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.0, 2.23606797749979, 2.8284271247461903, 1.4142135623730951, 1.0, 4.679631753336632, 1.0]


Save a new version of the predicted label that has a labelling index of 2 so that it appears as a different color in itksnap

In [ ]:
inference: Scan = ensemble_out_dataset[0]
label_path = inference.label_path
new_label_path = label_path.with_name(f"{fm.nifti_name(label_path)}_val2.nii.gz")
utils.set_label_value(label_path, new_label_path, 2)

PosixPath('/home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble_val2.nii.gz')

In [10]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.label_path
    print("itksnap", "-g", flair, "-o", t1, "-s", prediction, "-s", manual)

itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1033/ses-20171117/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1033/ses-20171117/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment1/ensemble_output/sub-ms1033/ses-20171117/flair.t1_ensemble.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1152/ses-20170529/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1152/ses-20170529/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment1/ensemble_output/sub-ms1152/ses-20170529/flair.t1_ensemble.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1163/ses-20180907/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1163/ses-20180907/t1.nii.gz -s /home/s

Produce commands to open images and labels in freeview

In [1]:
image_paths = ["one", "twp", "three"]
step1 = " -g ".join(image_paths)
step1.split(" ")

['one', '-g', 'twp', '-g', 'three']

In [ ]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.label_path
    print("freeview", flair, t1, f"{prediction}:colormap=heat", f"{manual}:colormap=heat")

freeview /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/pineal-CH.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1029/ses-20170816/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/pineal-SRS.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1188/ses-20200720/flair.t1_ensemble.nii.gz:colormap=heat